Paired with scrapeEarthquakes, these are exercises in scraping data from websites. I wanted two so I could bump them against each other to play with scikit-learn.

In [3]:
import pandas as pd
import re
import sys
from urllib import request
from bs4 import BeautifulSoup
def is_volcano_entry(table_row):
    cells = table_row.findAll("td")
    if len(cells) > 5:
        return True
    else:
        return False
def get_all_volcanoes(html_soup):
    volcanoes = []
    all_rows_in_html_page = html_soup.findAll("tr")
    #sys.stderr.write(repr(html_soup)+'\n')
    for table_row in all_rows_in_html_page:
        if is_volcano_entry(table_row):
            row_cells = table_row.findAll("td")
            volcano_entry = {
                "name":row_cells[0].text.strip(),
                "country":row_cells[1].text.strip(),
                "type":row_cells[2].text.strip(),
                "latitude":row_cells[3].text.strip(),
                "longitude":row_cells[4].text.strip(),
                "elevation":row_cells[5].text.strip()
                }
            volcanoes.append(volcano_entry)
    return volcanoes
html = request.urlopen("http://volcano.oregonstate.edu/volcano_table")
html_soup = BeautifulSoup(html, 'html.parser')
volcano_list = get_all_volcanoes(html_soup)    
indf = pd.DataFrame(volcano_list)
sys.stderr.write(repr(indf.head()) +'\n')
df = indf.convert_objects(convert_numeric=True)
# This says if latitude > 0 then set Hemisphere = 'Northern'
df.loc[df.latitude > 0, 'Hemisphere'] = 'Northern'
df.loc[df.latitude < 0, 'Hemisphere'] = 'Southern'
# This says if longitude < 0 then set EW Hemisphere = -1
df.loc[df.longitude < 0, 'EW Hemisphere'] = 'Western'
df.loc[df.longitude >=0, 'EW Hemisphere'] = 'Eastern'
# Japanese volcanoes
df.loc[df.country == 'Japan',['latitude','longitude']]
# dataframe sorted by proximity to Greenwich Meridian (long=0)
df.loc[(df.longitude-0.).abs().argsort()]


     country elevation latitude longitude             name           type
0      Japan       641    34.50    131.60              Abu         Shield
1      Chile      6046   -23.30    -67.62       Acamarachi  Stratovolcano
2  Guatemala      3976    14.50    -90.88       Acatenango  Stratovolcano
3     Turkey      1689    38.57     34.52  Acigöl-Nevsehir        Caldera
4        USA      3742    46.21   -121.49            Adams  Stratovolcano


,country,elevation,latitude,longitude,name,type,Hemisphere,EW Hemisphere
79,France,1464,45.78,2.97,Chaine des Puys,Dome,Northern,Eastern
239,Australia,230,-53.03,72.60,McDonald Islands,Complex Volcano,Southern,Eastern
65,Norway,2277,71.10,-8.20,Beerenberg,Stratovolcano,Northern,Western
72,Cameroon,4095,4.20,9.17,Cameroon (Mount),Stratovolcano,Northern,Eastern
276,United States,1220,NaN,NaN,Oahu,Shield,NaN,NaN
387,Guatemala,3158,14.61,-91.19,Tolimán,Stratovolcano,Northern,Western
26,Italy,949,41.73,12.70,Alban Hills,Caldera,Northern,Eastern
207,Papua New Guinea,1680,-8.95,148.15,Lamington,Stratovolcano,Southern,Eastern
119,USA,1920,55.08,-162.81,Frosty Peak,Stratovolcano,Northern,Western
42,South Atlantic Ocean,858,-7.95,-14.37,Ascensión,Stratovolcano,Southern,Western


In [4]:
df.columns

Index(['country', 'elevation', 'latitude', 'longitude', 'name', 'type', 'Hemisphere', 'EW Hemisphere'], dtype='object')

In [32]:
df.loc[:,'f'] = abs(df['longitude'])
df.sort('f').drop('f', axis=1)

,country,elevation,latitude,longitude,name,type,Hemisphere,EW Hemisphere
79,France,1464,45.78,2.97,Chaine des Puys,Dome,Northern,Eastern
241,EthiopiaKenya,1067,37.42,4.08,Mega Basalt Field,Pyroclastic Cone,Northern,Eastern
65,Norway,2277,71.10,-8.20,Beerenberg,Stratovolcano,Northern,Western
72,Cameroon,4095,4.20,9.17,Cameroon (Mount),Stratovolcano,Northern,Eastern
280,Cameroon,3011,6.25,10.50,Oku Volcanic Field,Stratovolcano,Northern,Eastern
391,United Kingdom,2060,-37.09,-12.28,Tristan da Cunha,Shield,Southern,Western
26,Italy,949,41.73,12.70,Alban Hills,Caldera,Northern,Eastern
209,Spain,670,29.03,-13.63,Lanzarote,Fissure vents,Northern,Western
121,Spain,529,28.36,-14.02,Fuerteventura,Fissure vents,Northern,Western
42,South Atlantic Ocean,858,-7.95,-14.37,Ascensión,Stratovolcano,Southern,Western


In [18]:
df.iloc[337,1:5]

elevation             1496
latitude             48.09
longitude            153.2
name         Sarychev Peak
Name: 337, dtype: object